# Braindecode CNN training and prediction 
This notebook loads epoched dataframes and trains a braindecode models on the two possible classes. Both trialwise and cropped are implemented. Saving the models is easy and can be imported to a different notebook for further analysis. 

In [13]:
%matplotlib inline
from helperFunctions import *
from constants import *
from dataAnalysisFunctions import getSEM, getCleanedSignal, getIntervals, getPowerRatio
import pandas as pd

from featureBuilder import featureBuilder
import random

random.seed(1)

import warnings
warnings.filterwarnings("ignore")




In [14]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

#Import svm model
from sklearn import svm


In [15]:
from braindecode.datautil.signal_target import SignalAndTarget

from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from torch import nn
from braindecode.torch_ext.util import set_random_seeds

from braindecode.torch_ext.optimizers import AdamW

import torch.nn.functional as F


## Build dataframes and variables needed

In [16]:
# Load data from memory
foldername='P001'
filename='part_P001_block_S004'
#filtered_data = loadData(datatype='filtered_data', foldername=foldername, filename=filename)
#filtered_cleaned_data = loadData(datatype='filtered_cleaned_data', foldername=foldername, filename=filename)
df = loadData(datatype='dataframe_filtered', foldername=foldername, filename=filename)


In [17]:
# Load data from memory
foldername='P001'
filename='part_P001_block_S005'
#filtered_data = loadData(datatype='filtered_data', foldername=foldername, filename=filename)
#filtered_cleaned_data = loadData(datatype='filtered_cleaned_data', foldername=foldername, filename=filename)
df2 = loadData(datatype='dataframe_filtered', foldername=foldername, filename=filename)


In [18]:
def getStateDF(df):
    # Select the trials needed to classify
    focused_state = df["trial_time"] == 0.7
    unfocused_state = df["trial_time"] > 0.9
    med_state = ((df["trial_time"] > 0.7) & (df["trial_time"] <= 0.9))

    focused_df = df[focused_state]
    unfocused_df = df[unfocused_state]
    med_df = df[med_state]
    
    return focused_df, unfocused_df, med_df

In [19]:
focused_df, unfocused_df, med_df = getStateDF(df)
focused_df2, unfocused_df2, med_df2 = getStateDF(df2)

In [21]:
focused_df = focused_df.append(focused_df2)
unfocused_df = unfocused_df.append(unfocused_df2)

In [22]:
# Create lists of just the epochs and corresponding focus level
data_type = "data_extended"
focused_epochs = []
focused_y = []

for idx, row in focused_df.iterrows(): 
    focused_epochs.append(np.array(row[data_type][StreamType.EEG.value][StreamType.DATA.value]))
    focused_y.append(1)

unfocused_epochs = []
unfocused_y = []
for idx, row in unfocused_df.iterrows(): 
    unfocused_epochs.append(np.array(row[data_type][StreamType.EEG.value][StreamType.DATA.value]))
    unfocused_y.append(0)
    
print("focused:", len(focused_y), "unfocused:", len(unfocused_y))
    

focused: 423 unfocused: 601


In [23]:
def zipShuffle(A, B):
    C = list(zip(A, B))
    random.shuffle(C)
    A, B = zip(*C)
    return np.array(A), np.array(B)


In [24]:
# Make the two sets the same length
focused_epochs = tidyEEGList(np.array(focused_epochs))
unfocused_epochs = tidyEEGList(np.array(unfocused_epochs))

focused_epochs, focused_y = zipShuffle(focused_epochs, focused_y)
unfocused_epochs, unfocused_y = zipShuffle(unfocused_epochs, unfocused_y)

num_per_type = min(len(focused_y), len(unfocused_y))

focused_epochs, focused_y = focused_epochs[:num_per_type], focused_y[:num_per_type]
unfocused_epochs, unfocused_y = unfocused_epochs[:num_per_type], unfocused_y[:num_per_type]

In [25]:
# Create X and y matrices 
X = np.array(list(focused_epochs) + list(unfocused_epochs))
y = np.array(list(focused_y) + list(unfocused_y))

X = tidyEEGList(X)
X = np.array([np.transpose(X[i]) for i in range(len(X))])

X, y = zipShuffle(X, y)
len(y)

846

In [27]:
# Split data up
percent_train = 0.7
num_train = int(len(X) * percent_train)
percent_valid = 0.2
num_valid = int(len(X) * percent_valid)

train_set = SignalAndTarget(X[:num_train], y[:num_train])
valid_set = SignalAndTarget(X[num_train:num_train + num_valid], y[num_train:num_train + num_valid])
test_set = SignalAndTarget(X[num_train + num_valid:], y=y[num_train + num_valid:])


In [45]:
test_set.X.shape

(776, 8, 546)

## Trialwise Decoding

In [31]:
# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
cuda = False
set_random_seeds(seed=20170629, cuda=cuda)
n_classes = 2
in_chans = train_set.X.shape[1]
# final_conv_length = auto ensures we only get a single output in the time dimension
model_t = ShallowFBCSPNet(in_chans=in_chans, n_classes=n_classes,
                        input_time_length=train_set.X.shape[2], 
                        #filter_time_length=4,
                        final_conv_length='auto')
if cuda:
    model_t.cuda()
    
#optimizer = AdamW(model.parameters(), lr=1*0.01, weight_decay=0.5*0.001) # these are good values for the deep model
optimizer = AdamW(model_t.parameters(), lr=0.0625 * 0.01, weight_decay=0)
model_t.compile(loss=F.nll_loss, optimizer=optimizer, iterator_seed=1,)

model_t.fit(train_set.X, train_set.y, epochs=30, batch_size=64, scheduler='cosine',
         validation_data=(valid_set.X, valid_set.y),)

model_t.epochs_df


,train_loss,valid_loss,train_misclass,valid_misclass,runtime
0,1.382881,1.455629,0.462838,0.473373,0.000000
1,0.655319,0.691541,0.364865,0.402367,5.445997
2,0.682284,0.848024,0.383446,0.437870,5.464340
3,0.604569,0.733885,0.295608,0.414201,5.439598
4,0.632555,0.874861,0.337838,0.455621,5.492461
5,0.534902,0.657645,0.278716,0.355030,5.424477
6,0.564443,0.808533,0.290541,0.420118,5.366806
7,0.518813,0.743238,0.258446,0.402367,5.678066
8,0.520349,0.806382,0.246622,0.402367,5.637397
9,0.480389,0.659013,0.244932,0.360947,5.329871


In [32]:
test_set = SignalAndTarget(X[70:], y=y[70:])

model_t.evaluate(test_set.X, test_set.y)

{'loss': 0.4951760470867157,
 'misclass': 0.23840206185567014,
 'runtime': 0.0008199214935302734}

In [33]:
pred_train_y = model_t.predict_classes(train_set.X)

sum(train_set.y == pred_train_y) / len(pred_train_y)

0.8243243243243243

In [34]:
pred_valid_y = model_t.predict_classes(valid_set.X)

sum(valid_set.y == pred_valid_y) / len(pred_valid_y)

0.6627218934911243

In [35]:
pred_test_y = model_t.predict_classes(test_set.X)

sum(test_set.y == pred_test_y) / len(pred_test_y)

0.7615979381443299

In [44]:
# import torch
# torch.save(model_t, "model_t_combined_filtered.pickle")
# torch.save(model_c, "model_c_combined_filtered.pickle")

## Cropped Decoding

In [36]:
# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
cuda = False
set_random_seeds(seed=20170629, cuda=cuda)
n_classes = 2
in_chans = train_set.X.shape[1]
# final_conv_length = auto ensures we only get a single output in the time dimension
model_c = ShallowFBCSPNet(in_chans=in_chans, n_classes=n_classes,
                        input_time_length=None, 
                        final_conv_length=12)
if cuda:
    model_c.cuda()
    


In [37]:
#optimizer = AdamW(model.parameters(), lr=1*0.01, weight_decay=0.5*0.001) # these are good values for the deep model
optimizer = AdamW(model_c.parameters(), lr=0.0625 * 0.01, weight_decay=0)
model_c.compile(loss=F.nll_loss, optimizer=optimizer,  iterator_seed=1, cropped=True)


In [38]:
input_time_length = train_set.X.shape[2]
model_c.fit(train_set.X, train_set.y, epochs=30, batch_size=64, scheduler='cosine',
          input_time_length=input_time_length,
         validation_data=(valid_set.X, valid_set.y),)

model_c.epochs_df


,train_loss,valid_loss,train_misclass,valid_misclass,runtime
0,1.764594,1.809530,0.501689,0.497041,0.000000
1,0.694437,0.766192,0.375000,0.437870,10.356950
2,0.677484,0.784519,0.361486,0.461538,10.888689
3,0.643068,0.734710,0.332770,0.437870,11.013071
4,0.633955,0.737799,0.319257,0.426036,10.933045
5,0.607242,0.720494,0.300676,0.402367,10.157255
6,0.608859,0.722492,0.305743,0.384615,10.048883
7,0.598556,0.695825,0.302365,0.378698,10.269390
8,0.591692,0.717708,0.297297,0.414201,10.483545
9,0.583772,0.681220,0.282095,0.378698,10.065620


In [39]:
test_set = SignalAndTarget(X[70:], y=y[70:])

model_c.evaluate(test_set.X, test_set.y)

{'loss': 0.5938166975975037,
 'misclass': 0.2860824742268041,
 'runtime': 0.0004119873046875}

In [40]:
pred_train_y = model_c.predict_classes(train_set.X)

sum(train_set.y == pred_train_y) / len(pred_train_y)

0.7449324324324325

In [41]:
pred_valid_y = model_c.predict_classes(valid_set.X)

sum(valid_set.y == pred_valid_y) / len(pred_valid_y)

0.6272189349112426

In [42]:
pred_test_y = model_c.predict_classes(test_set.X)

sum(test_set.y == pred_test_y) / len(pred_test_y)

0.7139175257731959

In [ ]:
features = model.network

In [ ]:
features

In [ ]:
conv_layer_1 = features[1]

In [ ]:
conv_layer_1.weight.data.shape

In [ ]:
conv_layer_1_numpy = conv_layer_1.weight.data.numpy() 

In [ ]:
conv_layer_1_numpy = conv_layer_1_numpy.reshape(40, 25)


In [ ]:
import seaborn as sns; sns.set()

In [ ]:
plt.figure(figsize=(20,10))

ax = sns.heatmap(conv_layer_1_numpy)
plt.xlabel("Kernel")
plt.ylabel("Output")
plt.title("Weights of first layer")
plt.show()

In [ ]:
conv_layer_2 = features[2]

In [ ]:
conv_layer_2.weight.data.shape

In [ ]:
conv_layer_2_numpy = conv_layer_2.weight.data.numpy()

In [ ]:
conv_layer_2_numpy = conv_layer_2_numpy.reshape(40, 40, 8)

In [ ]:
for i in range(8): 
    ax = sns.heatmap(conv_layer_2_numpy[:,:,i])
    plt.show()


In [ ]:
r = np.corrcoef(conv_layer_2_numpy[:,:,0], conv_layer_2_numpy[:,:,1])

In [ ]:
r.shape

In [ ]:
ax = sns.heatmap(r)

# Doesn't work 

for p in model.parameters():
    print(p)

from torchvision import utils

def visTensor(tensor, ch=0, allkernels=False, nrow=8, padding=1): 
    n,c,w,h = tensor.shape

    if allkernels: tensor = tensor.view(n*c, -1, w, h)
    elif c != 3: tensor = tensor[:,ch,:,:].unsqueeze(dim=1)

    rows = np.min((tensor.shape[0] // nrow + 1, 64))    
    grid = utils.make_grid(tensor, nrow=nrow, normalize=True, padding=padding)
    plt.figure( figsize=(nrow,rows) )
    plt.imshow(grid.numpy().transpose((1, 2, 0)))

params = []
for i,p in enumerate(model.parameters()):
    #print(i)
    #print(p)
    #params.append(p)
    visTensor(p)


def plot_weights(features, layer_num, single_channel = True, collated = False):
  
    #extracting the model features at the particular layer number
    layer = features[layer_num]
  
    #checking whether the layer is convolution layer or not 
    if isinstance(layer, nn.Conv2d):
        #getting the weight tensor data
        weight_tensor = features[layer_num].weight.data
        return weight_tensor
        
        #grid = vutils.make_grid(weight_tensor)
        #show(grid)
        #         if single_channel:
        #             if collated:
        #                 plot_filters_single_channel_big(weight_tensor)
        #             else:
        #                 plot_filters_single_channel(weight_tensor)

        #         else:
        #             if weight_tensor.shape[1] == 3:
        #                 plot_filters_multi_channel(weight_tensor)
        #             else:
        #                 print("Can only plot weights with three channels with single channel = False")

    else:
        print("Can only visualize layers which are convolutional")

import torchvision.utils as vutils
def show(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')
